In [34]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro
import scipy.stats as stats

In [35]:
def AB_test(dataframe, group, target):

    #splitting groups
    groupA=dataframe[dataframe[group]==1][target] #holiday
    groupB=dataframe[dataframe[group]==0][target] #not holiday

    #checking distributin of groups using shapiro 
    #H0: distribution is normal
    # pA=shapiro(groupA)[1]
    # pB=shapiro(groupB)[1]

    # if (pA >=0.05) & (pB>=0.05):
        #both are normally distributed so we use parametric test

    #checking homogeneity of variances using levene test
    #H0: there is homogeneity of variances
    leveneTest_p = stats.levene(groupA, groupB)[1]

    if leveneTest_p<0.05:
        #heterogeneous

        #t test
        #H0: M1 == M2
        p=stats.ttest_ind(groupA, groupB, equal_var=False)[1]
    else:
        #homogeneity

        #ttest
        #H0:M1==M2
        p=stats.ttest_ind(groupA, groupB, equal_var=True)[1]
    # else:
    #     #non-parametric test

    #     #Mann-Whitney U test
    #     # H0: M1 == M2
    #     p=stats.mannwhitneyu(groupA, groupB)[1]

    group = [group]
    p = [p]
    # pA = [pA]
    # pB = [pB]

    AB = pd.DataFrame({
    "Feature": group,
    "p-value": p,
    #"Test": np.where((np.array(pA) == False) & (np.array(pB) == False), "t-Test (p)", "Mann-Whitney U (nonp)"),
    "Hypothesis": np.where(np.array(p) >= 0.05, "Fail to Reject H0", "Reject H0"),
    "Comment": np.where(np.array(p) >= 0.05, "A/B groups are similar", "A/B groups are not similar"),
    "GroupA_mean": np.mean(groupA),
    "GroupB_mean": np.mean(groupB),
    "GroupA_median": np.median(groupA),
    "GroupB_median": np.median(groupB)
    })
    return AB

In [36]:
df=pd.read_csv(r'novi_datasetovi\train_test_v1.csv', parse_dates=['date'])
df=df.drop(columns='Unnamed: 0')
df.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,...,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer Grito de Independencia,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0


In [37]:
df.shape

(3036528, 54)

In [38]:
train=df[df['date']<'2017-08-16']
test=df[df['date']>= '2017-08-16']

In [39]:
#A/B Testing

cols = train.columns[train.columns.str.startswith("N ")].tolist() + train.columns[train.columns.str.startswith("R ")].tolist() + train.columns[train.columns.str.startswith("L ")].tolist()+ train.columns[train.columns.str.startswith("work_day")].tolist()
ab = []
for i in cols:
    ab.append(AB_test(dataframe=train, group = i, target = "sales"))
ab = pd.concat(ab)
ab

,Feature,p-value,Hypothesis,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,N Batalla de Pichincha,5.181154e-03,Reject H0,A/B groups are not similar,391.726745,356.824556,12.0,11.0
0,N Black Friday,6.689129e-01,Fail to Reject H0,A/B groups are similar,363.360708,356.916486,17.0,11.0
0,N Carnaval,2.323565e-03,Reject H0,A/B groups are not similar,332.882573,357.071237,10.0,11.0
0,N Cyber Monday,4.249429e-06,Reject H0,A/B groups are not similar,436.220250,356.786766,17.0,11.0
0,N Dia de Difuntos,6.682849e-08,Reject H0,A/B groups are not similar,431.034644,356.707777,16.0,11.0
0,N Dia de la Madre,4.930530e-06,Reject H0,A/B groups are not similar,397.836026,356.684149,13.0,11.0
0,N Dia del Trabajo,4.023208e-14,Reject H0,A/B groups are not similar,481.446180,356.558010,13.0,11.0
0,N Futbol,1.659731e-14,Reject H0,A/B groups are not similar,310.965144,357.312335,5.0,11.0
0,N Independencia de Cuenca,4.611872e-12,Reject H0,A/B groups are not similar,477.446607,356.641672,13.0,11.0
0,N Independencia de Guayaquil,2.608445e-06,Reject H0,A/B groups are not similar,430.354990,356.753528,16.0,11.0


In [40]:
#stores don't work on new years (except store 25,36) and christmass
train[(((train['date'].dt.month == 12) & (train['date'].dt.day == 25))| ((train['date'].dt.month == 1) & (train['date'].dt.day == 1) & (~(train['store_nbr'].isin([25,36]))))) & (train['sales']!=0)]

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,...,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer Grito de Independencia,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions


In [41]:
#again with local regional and national columns

df2=df.copy()

In [42]:
#PREDUGO

# nat_cols = [col for col in train2.columns if col.startswith('N ')]
# reg_cols = [col for col in train2.columns if col.startswith('R ')]
# loc_cols = [col for col in train2.columns if col.startswith('L ')]

# # ažuriranje kolona za praznike
# def check_for_holidays(row):
#     if row[nat_cols].sum() > 0:
#         row['national_holiday'] = 1
    
#     if row[reg_cols].sum() > 0:
#         for col in reg_cols:
#             if row[col] == 1 and row['state'] in col:
#                 row['regional_holiday'] = 1
#                 break
    
#     if row[loc_cols].sum() > 0:
#         for col in loc_cols:
#             if row[col] == 1 and row['city'] in col:
#                 row['local_holiday'] = 1
#                 break
    
#     return row

# train2 = train2.apply(check_for_holidays, axis=1)

In [43]:
local_holidays=pd.read_csv(r'holiday_datasets/local_holidays.csv', parse_dates=['date'])
regional_holidays=pd.read_csv(r'holiday_datasets/regional_holidays.csv',parse_dates=['date'])
national_holidays=pd.read_csv(r'holiday_datasets/national_holidays.csv', parse_dates=['date'])

In [44]:
local_holidays['name']=local_holidays['local_holidays'].copy()
local_holidays=local_holidays.drop(columns=['local_holidays'])
local_holidays['local_holidays']=1
local_holidays2=local_holidays.copy()
local_holidays=local_holidays.drop(columns=['name'])

regional_holidays['name']=regional_holidays['regional_holidays'].copy()
regional_holidays=regional_holidays.drop(columns=['regional_holidays'])
regional_holidays['regional_holidays']=1
regional_holidays2=regional_holidays.copy()
regional_holidays=regional_holidays.drop(columns=['name'])

national_holidays=national_holidays.drop(columns=['national_holidays'])
national_holidays['national_holidays']=1

In [45]:
local_holidays=local_holidays.drop_duplicates()
regional_holidays=regional_holidays.drop_duplicates()
national_holidays=national_holidays.drop_duplicates()

In [46]:
local_holidays2=local_holidays2.drop_duplicates()
regional_holidays2=regional_holidays2.drop_duplicates()

In [47]:
df2=df2.merge(local_holidays, how='left', on=['date', 'city'])
df2['local_holidays']=df2['local_holidays'].fillna(0)
print(df2.shape)

df2=df2.merge(regional_holidays, how='left', on=['date', 'state'])
df2['regional_holidays']=df2['regional_holidays'].fillna(0)
print(df2.shape)

df2=df2.merge(national_holidays, how='left', on=['date'])
df2['national_holidays']=df2['national_holidays'].fillna(0)
print(df2.shape)

df2['national_holidays'] = df2['national_holidays'].astype(int)
df2['regional_holidays'] = df2['regional_holidays'].astype(int)
df2['local_holidays'] = df2['local_holidays'].astype(int)
print(df2.shape)

df2.head()

(3036528, 55)


(3036528, 56)
(3036528, 57)
(3036528, 57)


,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,...,N Navidad,N Primer Grito de Independencia,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1


In [48]:
df2.to_csv('novi_datasetovi/train_test_v3.csv', index=False)

In [49]:
train2=df2[df2['date']<'2017-08-16']
test2=df2[df2['date']>= '2017-08-16']

In [50]:
local_holidays2.head()

,date,city,name,local_holidays
0,2012-03-02,Manta,Fundacion de Manta,1
1,2012-04-12,Cuenca,Fundacion de Cuenca,1
2,2012-04-14,Libertad,Cantonizacion de Libertad,1
3,2012-04-21,Riobamba,Cantonizacion de Riobamba,1
4,2012-05-12,Puyo,Cantonizacion del Puyo,1


In [51]:
def AB_test_local_holidays(dataframe, group, target):

    s=local_holidays2['name']
    city=local_holidays2['L '+s ==group].city.values[0]
    #splitting groups
    groupA=dataframe[(dataframe['city']==city)&(dataframe[group]==1)&(dataframe['national_holidays']==0)&(dataframe['regional_holidays']==0)][target] #holiday
    groupB=dataframe[(dataframe['city']==city)&(dataframe[group]==0)&(dataframe['national_holidays']==0)&(dataframe['regional_holidays']==0)][target] #not holiday


    # checking distributin of groups using shapiro 
    # H0: distribution is normal
    # pA=shapiro(groupA)[1]
    # pB=shapiro(groupB)[1]

    # if (pA >=0.05) & (pB>=0.05):
    #     both are normally distributed so we use parametric test

    # checking homogeneity of variances using levene test
    # H0: there is homogeneity of variances
    leveneTest_p = stats.levene(groupA, groupB)[1]

    if leveneTest_p<0.05:
        #heterogeneous

        #t test
        #H0: M1 == M2
        p=stats.ttest_ind(groupA, groupB, equal_var=False)[1]
    else:
        #homogeneity

        #ttest
        #H0:M1==M2
        p=stats.ttest_ind(groupA, groupB, equal_var=True)[1]
    # else:
    #     #non-parametric test

    #     #Mann-Whitney U test
    #     # H0: M1 == M2
    #     p=stats.mannwhitneyu(groupA, groupB)[1]

    group = [group]
    p = [p]
    # pA = [pA]
    # pB = [pB]

    AB = pd.DataFrame({
    "Feature": group,
    "p-value": p,
    #"Test": np.where((np.array(pA) == False) & (np.array(pB) == False), "t-Test (p)", "Mann-Whitney U (nonp)"),
    "Hypothesis": np.where(np.array(p) >= 0.05, "Fail to Reject H0", "Reject H0"),
    "Comment": np.where(np.array(p) >= 0.05, "A/B groups are similar", "A/B groups are not similar"),
    "GroupA_mean": np.mean(groupA),
    "GroupB_mean": np.mean(groupB),
    "GroupA_median": np.median(groupA),
    "GroupB_median": np.median(groupB)
    })
    return AB

In [52]:
cols =train2.columns[train2.columns.str.startswith("L ")].tolist()
ab_loc = []
for i in cols:
    ab_loc.append(AB_test_local_holidays(dataframe=train2, group = i, target = "sales"))
ab_loc = pd.concat(ab_loc)
ab_loc

c:\Users\Snezana\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Snezana\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Feature,p-value,Hypothesis,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,L Cantonizacion de Cayambe,5.771010e-01,Fail to Reject H0,A/B groups are similar,445.602558,503.840132,23.000000,15.0
0,L Cantonizacion de El Carmen,6.892550e-01,Fail to Reject H0,A/B groups are similar,214.438109,196.498199,9.000000,6.0
0,L Cantonizacion de Guaranda,2.913841e-01,Fail to Reject H0,A/B groups are similar,178.015915,231.550912,6.000000,8.0
0,L Cantonizacion de Latacunga,9.092000e-01,Fail to Reject H0,A/B groups are similar,192.490970,188.764269,11.000000,6.0
0,L Cantonizacion de Libertad,1.534507e-01,Fail to Reject H0,A/B groups are similar,180.356318,270.480139,0.000000,5.0
0,L Cantonizacion de Quevedo,3.507754e-01,Fail to Reject H0,A/B groups are similar,200.778636,252.130131,9.000000,9.0
0,L Cantonizacion de Riobamba,4.981113e-01,Fail to Reject H0,A/B groups are similar,171.422417,202.831494,4.000000,6.0
0,L Cantonizacion de Salinas,NaN,Reject H0,A/B groups are not similar,NaN,196.546649,NaN,8.0
0,L Cantonizacion del Puyo,6.445678e-01,Fail to Reject H0,A/B groups are similar,56.501323,70.698262,0.000000,0.0
0,L Fundacion de Ambato,5.918035e-01,Fail to Reject H0,A/B groups are similar,325.854186,357.439475,19.500000,21.0


In [53]:
def AB_test_local_holidays_mw(dataframe, group, target):

    s=local_holidays2['name']
    city=local_holidays['L '+s ==group].city.values[0]
    #splitting groups
    groupA=dataframe[(dataframe['city']==city)&(dataframe[group]==1)&(dataframe['national_holidays']==0)&(dataframe['regional_holidays']==0)][target] #holiday
    groupB=dataframe[(dataframe['city']==city)&(dataframe[group]==0)&(dataframe['national_holidays']==0)&(dataframe['regional_holidays']==0)][target] #not holiday

    print(group)
    print('groupA ', groupA.shape)
    print('groupB ', groupB.shape)
    

    #checking distributin of groups using shapiro 
    #H0: distribution is normal
    if groupA.empty or groupB.empty:
        AB = pd.DataFrame(columns=["Feature", "p-value", "Test", "Hypothesis", "Comment", 
                                    "GroupA_mean", "GroupB_mean", "GroupA_median", "GroupB_median"])
        return AB
    pA=shapiro(groupA)[1]
    pB=shapiro(groupB)[1]

    if (pA >=0.05) & (pB>=0.05):
        #both are normally distributed so we use parametric test

        #checking homogeneity of variances using levene test
        #H0: there is homogeneity of variances
        leveneTest_p = stats.levene(groupA, groupB)[1]

        if leveneTest_p<0.05:
            #heterogeneous

            #t test
            #H0: M1 == M2
            p=stats.ttest_ind(groupA, groupB, equal_var=False)[1]
        else:
            #homogeneity

            #ttest
            #H0:M1==M2
            p=stats.ttest_ind(groupA, groupB, equal_var=True)[1]
    else:
        #non-parametric test

        #Mann-Whitney U test
        # H0: M1 == M2
        p=stats.mannwhitneyu(groupA, groupB)[1]

    group = [group]
    p = [p]
    pA = [pA]
    pB = [pB]

    AB = pd.DataFrame({
    "Feature": group,
    "p-value": p,
    "Test": np.where((np.array(pA) == False) & (np.array(pB) == False), "t-Test (p)", "Mann-Whitney U (nonp)"),
    "Hypothesis": np.where(np.array(p) >= 0.05, "Fail to Reject H0", "Reject H0"),
    "Comment": np.where(np.array(p) >= 0.05, "A/B groups are similar", "A/B groups are not similar"),
    "GroupA_mean": np.mean(groupA),
    "GroupB_mean": np.mean(groupB),
    "GroupA_median": np.median(groupA),
    "GroupB_median": np.median(groupB)
    })
    return AB

In [54]:
cols =train2.columns[train2.columns.str.startswith("L ")].tolist()
ab_loc2 = []
for i in cols:
    ab_loc2.append(AB_test_local_holidays_mw(dataframe=train2, group = i, target = "sales"))
ab_loc2 = pd.concat(ab_loc2)
ab_loc2

L Cantonizacion de Cayambe
groupA  (165,)
groupB  (51084,)


c:\Users\Snezana\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


L Cantonizacion de El Carmen
groupA  (165,)
groupB  (51084,)
L Cantonizacion de Guaranda
groupA  (165,)
groupB  (51084,)


L Cantonizacion de Latacunga
groupA  (264,)
groupB  (101904,)
L Cantonizacion de Libertad
groupA  (132,)
groupB  (51117,)
L Cantonizacion de Quevedo
groupA  (132,)
groupB  (51117,)
L Cantonizacion de Riobamba
groupA  (132,)
groupB  (51117,)
L Cantonizacion de Salinas
groupA  (0,)
groupB  (51117,)
L Cantonizacion del Puyo
groupA  (99,)
groupB  (51150,)


c:\Users\Snezana\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


L Fundacion de Ambato
groupA  (264,)
groupB  (102234,)
L Fundacion de Cuenca
groupA  (495,)
groupB  (153252,)
L Fundacion de Esmeraldas
groupA  (165,)
groupB  (51084,)
L Fundacion de Guayaquil
groupA  (2376,)
groupB  (407616,)
L Fundacion de Ibarra
groupA  (132,)
groupB  (50985,)
L Fundacion de Loja
groupA  (132,)
groupB  (51117,)
L Fundacion de Machala
groupA  (264,)
groupB  (102234,)
L Fundacion de Manta
groupA  (330,)
groupB  (102168,)
L Fundacion de Quito
groupA  (4752,)
groupB  (917730,)
L Fundacion de Riobamba
groupA  (165,)
groupB  (51084,)
L Fundacion de Santo Domingo
groupA  (495,)
groupB  (152856,)
L Independencia de Ambato
groupA  (264,)
groupB  (102234,)
L Independencia de Guaranda
groupA  (132,)
groupB  (51117,)
L Independencia de Latacunga
groupA  (264,)
groupB  (101904,)


,Feature,p-value,Test,Hypothesis,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,L Cantonizacion de Cayambe,3.800485e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,445.602558,503.840132,23.000000,15.0
0,L Cantonizacion de El Carmen,7.610964e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,214.438109,196.498199,9.000000,6.0
0,L Cantonizacion de Guaranda,5.002666e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,178.015915,231.550912,6.000000,8.0
0,L Cantonizacion de Latacunga,1.112805e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,192.490970,188.764269,11.000000,6.0
0,L Cantonizacion de Libertad,1.059439e-03,Mann-Whitney U (nonp),Reject H0,A/B groups are not similar,180.356318,270.480139,0.000000,5.0
0,L Cantonizacion de Quevedo,5.904379e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,200.778636,252.130131,9.000000,9.0
0,L Cantonizacion de Riobamba,3.201984e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,171.422417,202.831494,4.000000,6.0
0,L Cantonizacion del Puyo,2.959183e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,56.501323,70.698262,0.000000,0.0
0,L Fundacion de Ambato,6.078569e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,325.854186,357.439475,19.500000,21.0
0,L Fundacion de Cuenca,1.776302e-01,Mann-Whitney U (nonp),Fail to Reject H0,A/B groups are similar,263.715962,292.431594,6.000000,8.0


In [55]:
def AB_test_regional_holidays(dataframe, group, target):

    s=regional_holidays2['name']
    state=regional_holidays2['R '+s==group].state.values[0]
    #splitting groups
    groupA=dataframe[(dataframe['state']==state)&(dataframe[group]==1)&(dataframe['national_holidays']==0)][target] #holiday
    groupB=dataframe[(dataframe['state']==state)&(dataframe[group]==0)&(dataframe['national_holidays']==0)][target] #not holiday


    # checking distributin of groups using shapiro 
    # H0: distribution is normal
    # pA=shapiro(groupA)[1]
    # pB=shapiro(groupB)[1]

    # if (pA >=0.05) & (pB>=0.05):
    #     both are normally distributed so we use parametric test

    # checking homogeneity of variances using levene test
    # H0: there is homogeneity of variances
    leveneTest_p = stats.levene(groupA, groupB)[1]

    if leveneTest_p<0.05:
        #heterogeneous

        #t test
        #H0: M1 == M2
        p=stats.ttest_ind(groupA, groupB, equal_var=False)[1]
    else:
        #homogeneity

        #ttest
        #H0:M1==M2
        p=stats.ttest_ind(groupA, groupB, equal_var=True)[1]
    # else:
    #     #non-parametric test

    #     #Mann-Whitney U test
    #     # H0: M1 == M2
    #     p=stats.mannwhitneyu(groupA, groupB)[1]

    group = [group]
    p = [p]
    # pA = [pA]
    # pB = [pB]

    AB = pd.DataFrame({
    "Feature": group,
    "p-value": p,
    #"Test": np.where((np.array(pA) == False) & (np.array(pB) == False), "t-Test (p)", "Mann-Whitney U (nonp)"),
    "Hypothesis": np.where(np.array(p) >= 0.05, "Fail to Reject H0", "Reject H0"),
    "Comment": np.where(np.array(p) >= 0.05, "A/B groups are similar", "A/B groups are not similar"),
    "GroupA_mean": np.mean(groupA),
    "GroupB_mean": np.mean(groupB),
    "GroupA_median": np.median(groupA),
    "GroupB_median": np.median(groupB)
    })
    return AB

In [56]:
cols =train2.columns[train2.columns.str.startswith("R ")].tolist()
ab_loc3 = []
for i in cols:
    ab_loc3.append(AB_test_regional_holidays(dataframe=train2, group = i, target = "sales"))
ab_loc3 = pd.concat(ab_loc3)
ab_loc3

,Feature,p-value,Hypothesis,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,R Provincializacion Santa Elena,0.760146,Fail to Reject H0,A/B groups are similar,182.504114,196.546649,10.5,8.0
0,R Provincializacion de Cotopaxi,0.121943,Fail to Reject H0,A/B groups are similar,234.051215,188.773899,5.0,6.0
0,R Provincializacion de Imbabura,0.758337,Fail to Reject H0,A/B groups are similar,190.604439,204.947631,11.5,8.0
0,R Provincializacion de Santo Domingo,0.084198,Fail to Reject H0,A/B groups are similar,264.017609,211.518059,6.0,4.0


In [58]:
regional_holidays2.head()

,date,state,name,regional_holidays
0,2012-04-01,Cotopaxi,Provincializacion de Cotopaxi,1
1,2012-06-25,Imbabura,Provincializacion de Imbabura,1
2,2012-11-06,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,1
3,2012-11-07,Santa Elena,Provincializacion Santa Elena,1
4,2013-04-01,Cotopaxi,Provincializacion de Cotopaxi,1


In [ ]:
pd.set_option('display.max_columns',None)

In [59]:
def AB_test_national_holidays(dataframe, group, target):

    #splitting groups
    groupA=dataframe[(dataframe[group]==1)][target] #holiday
    groupB=dataframe[(dataframe[group]==0)&(dataframe['national_holidays']==0)][target] #not holiday


    # checking distributin of groups using shapiro 
    # H0: distribution is normal
    # pA=shapiro(groupA)[1]
    # pB=shapiro(groupB)[1]

    # if (pA >=0.05) & (pB>=0.05):
    #     both are normally distributed so we use parametric test

    # checking homogeneity of variances using levene test
    # H0: there is homogeneity of variances
    leveneTest_p = stats.levene(groupA, groupB)[1]

    if leveneTest_p<0.05:
        #heterogeneous

        #t test
        #H0: M1 == M2
        p=stats.ttest_ind(groupA, groupB, equal_var=False)[1]
    else:
        #homogeneity

        #ttest
        #H0:M1==M2
        p=stats.ttest_ind(groupA, groupB, equal_var=True)[1]
    # else:
    #     #non-parametric test

    #     #Mann-Whitney U test
    #     # H0: M1 == M2
    #     p=stats.mannwhitneyu(groupA, groupB)[1]

    group = [group]
    p = [p]
    # pA = [pA]
    # pB = [pB]

    AB = pd.DataFrame({
    "Feature": group,
    "p-value": p,
    #"Test": np.where((np.array(pA) == False) & (np.array(pB) == False), "t-Test (p)", "Mann-Whitney U (nonp)"),
    "Hypothesis": np.where(np.array(p) >= 0.05, "Fail to Reject H0", "Reject H0"),
    "Comment": np.where(np.array(p) >= 0.05, "A/B groups are similar", "A/B groups are not similar"),
    "GroupA_mean": np.mean(groupA),
    "GroupB_mean": np.mean(groupB),
    "GroupA_median": np.median(groupA),
    "GroupB_median": np.median(groupB)
    })
    return AB

In [60]:
cols =train2.columns[train2.columns.str.startswith("N ")].tolist()
ab_loc4 = []
for i in cols:
    ab_loc4.append(AB_test_national_holidays(dataframe=train2, group = i, target = "sales"))
ab_loc4 = pd.concat(ab_loc4)
ab_loc4

,Feature,p-value,Hypothesis,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,N Batalla de Pichincha,1.648835e-03,Reject H0,A/B groups are not similar,391.726745,352.430783,12.0,11.0
0,N Black Friday,4.591238e-01,Fail to Reject H0,A/B groups are similar,363.360708,352.430783,17.0,11.0
0,N Carnaval,1.385534e-02,Reject H0,A/B groups are not similar,332.882573,352.430783,10.0,11.0
0,N Cyber Monday,1.232701e-06,Reject H0,A/B groups are not similar,436.220250,352.430783,17.0,11.0
0,N Dia de Difuntos,1.133768e-08,Reject H0,A/B groups are not similar,431.034644,352.430783,16.0,11.0
0,N Dia de la Madre,4.678908e-07,Reject H0,A/B groups are not similar,397.836026,352.430783,13.0,11.0
0,N Dia del Trabajo,5.764254e-15,Reject H0,A/B groups are not similar,481.446180,352.430783,13.0,11.0
0,N Futbol,6.612585e-12,Reject H0,A/B groups are not similar,310.965144,352.430783,5.0,11.0
0,N Independencia de Cuenca,8.240484e-13,Reject H0,A/B groups are not similar,477.446607,352.430783,13.0,11.0
0,N Independencia de Guayaquil,6.530250e-07,Reject H0,A/B groups are not similar,430.354990,352.430783,16.0,11.0


In [61]:
holidays_to_keep=['N Batalla de Pichincha','N Carnaval','N Cyber Monday','N Dia de Difuntos','N Dia de la Madre','N Dia del Trabajo','N Futbol','N Independencia de Cuenca','N Independencia de Guayaquil','N Navidad','N Primer dia del ano','N Terremoto Manabi','N Viernes Santo']

In [62]:
cols_to_drop = df2.columns[df2.columns.str.startswith("L ")].tolist() + df2.columns[df2.columns.str.startswith("R ")].tolist() #loc and reg
cols_to_drop += [col for col in df2.columns if col.startswith('N ') and col not in holidays_to_keep]

df2_filtered = df2.drop(columns=cols_to_drop)


In [63]:
df2_filtered.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,...,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,1,0,0,93.14,0.0,0,0,1


In [64]:
df2_filtered.to_csv(r'novi_datasetovi\train_test_v4.csv', index=False)